In [2]:
import scipy as sp
import pandas as pd
import numpy as np

In [3]:
# read data/aPCx_AA.mat
mat_data = sp.io.loadmat('data/aPCx_AA.mat', struct_as_record=False, squeeze_me=True)

# Convert to ndarray (extract the main data variable)
data = mat_data['espe']

In [4]:
# Check the keys in mat_data
print("Keys in mat_data:", list(mat_data.keys()))
print("\nData type:", type(data))
print("Data shape:", data.shape if hasattr(data, 'shape') else "No shape")
print("\nData dtype:", data.dtype)

Keys in mat_data: ['__header__', '__version__', '__globals__', 'espe']

Data type: <class 'numpy.ndarray'>
Data shape: (10,)

Data dtype: object


In [5]:
# Flatten key fields to a CSV-friendly table

def to_scalar(val, default=np.nan):
    try:
        arr = np.array(val)
        if arr.size == 1:
            return arr.item()
        return arr
    except Exception:
        return default

rows = []
for shank_idx, shank in enumerate(np.atleast_1d(data)):
    shank_struct = getattr(shank, 'shank', None)
    if shank_struct is None:
        continue
    for sua_idx, sua in enumerate(np.atleast_1d(shank_struct)):
        sua_struct = getattr(sua, 'SUA', None)
        if sua_struct is None:
            continue
        cells = getattr(sua_struct, 'cell', [])
        for cell_idx, cell in enumerate(np.atleast_1d(cells)):
            rows.append({
                'shank': shank_idx,
                'sua': sua_idx, # Single Unit Activity index
                'cell': cell_idx,
                'odor_count': len(np.atleast_1d(getattr(cell, 'odor', []))),
                'spike_snr': to_scalar(getattr(cell, 'spikeSNR', np.nan)),
                'isolation_distance': to_scalar(getattr(cell, 'isolationDistance', np.nan)),
                'l_ratio': to_scalar(getattr(cell, 'L_Ratio', np.nan)),
                'good': to_scalar(getattr(cell, 'good', np.nan)),
            })

df = pd.DataFrame(rows)
df.to_csv('data/csv/aPCx_AA_flat.csv', index=False)

In [6]:
# Save the processed .mat data in pickle format
import pickle
with open('data/pickle/aPCx_AA.pkl', 'wb') as f:
    pickle.dump(mat_data, f)

In [7]:
mat_data = sp.io.loadmat('data/aPCx_AA.mat')
data = mat_data['espe']

# Explore deeper - cell level
shank = data[0][0]
sua_array = shank[0]  # Get the SUA array
cell = sua_array[0][0]  # First cell in first SUA

print("Type of cell:", type(cell))
print("Cell dtype:", cell.dtype if hasattr(cell, 'dtype') else "N/A")
print("Cell shape:", cell.shape if hasattr(cell, 'shape') else "N/A")
print("Fields in cell:", cell.dtype.names if hasattr(cell, 'dtype') else "N/A")

# Look at each field
if hasattr(cell, 'dtype') and cell.dtype.names:
    for field in cell.dtype.names:
        val = cell[field]
        print(f"\n{field}: type={type(val).__name__}, shape={val.shape if hasattr(val, 'shape') else 'N/A'}")

Type of cell: <class 'numpy.void'>
Cell dtype: [('SUA', 'O')]
Cell shape: ()
Fields in cell: ('SUA',)

SUA: type=ndarray, shape=(1, 1)


In [8]:
cell['SUA']['cell'][0][0]['odor'][0][0]['spikeMatrix'][0][3]

<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 284 stored elements and shape (10, 10000)>

In [9]:
print(cell['SUA']['cell'][0][0]['odor'][0][0]['spikeMatrix'][0][5].shape)
print(cell['SUA']['cell'][0][0]['odor'][0][0]['spikeMatrix'][0][5])

(10, 10000)
<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 282 stored elements and shape (10, 10000)>
  Coords	Values
  (4, 29)	1
  (4, 38)	1
  (4, 58)	1
  (1, 190)	1
  (1, 368)	1
  (3, 384)	1
  (3, 451)	1
  (9, 480)	1
  (0, 562)	1
  (6, 591)	1
  (4, 616)	1
  (4, 627)	1
  (5, 638)	1
  (4, 647)	1
  (2, 654)	1
  (0, 667)	1
  (5, 667)	1
  (6, 671)	1
  (6, 678)	1
  (6, 689)	1
  (6, 705)	1
  (8, 756)	1
  (8, 767)	1
  (8, 840)	1
  (8, 853)	1
  :	:
  (0, 9102)	1
  (8, 9138)	1
  (1, 9146)	1
  (1, 9151)	1
  (8, 9164)	1
  (1, 9168)	1
  (8, 9173)	1
  (8, 9182)	1
  (1, 9266)	1
  (1, 9276)	1
  (1, 9415)	1
  (2, 9437)	1
  (3, 9510)	1
  (1, 9568)	1
  (1, 9595)	1
  (8, 9609)	1
  (7, 9629)	1
  (0, 9668)	1
  (4, 9697)	1
  (1, 9750)	1
  (9, 9757)	1
  (0, 9973)	1
  (4, 9975)	1
  (3, 9986)	1
  (8, 9992)	1


In [23]:
def spike_matrix_to_firing_rate(spike_matrix):
    spike_matrix = cell['SUA']['cell'][0][0]['odor'][0][0]['spikeMatrix'][0][3]
    firing_rate = spike_matrix.mean(axis=1) * 1000  # spikes per second
    return firing_rate

In [24]:
# Use the function on a sample spike matrix
spike_matrix = cell['SUA']['cell'][0][0]['odor'][0][0]['spikeMatrix'][0][0]
firing_rates = spike_matrix_to_firing_rate(spike_matrix)
print("Firing rates shape:", firing_rates.shape)
print("Firing rates:", firing_rates)    

Firing rates shape: (10, 1)
Firing rates: [[3.2]
 [3.2]
 [3.4]
 [3.1]
 [3.2]
 [2. ]
 [2.6]
 [2.6]
 [2.3]
 [2.8]]
